In [41]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
!export OPENAI_API_KEY="4"
 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
%env OPENAI_API_KEY="g"

env: OPENAI_API_KEY="g"


In [43]:
import os
import json
import seaborn as sns
from collections import Counter
import openai
import numpy as np
import pandas as pd
from explorer.utils import load_states_from_dir
from games.ultimatum.ultimatum_multi_turn.game import MultiTurnUltimatumGame
import itertools
from collections import defaultdict
import os
import json
import openai
import numpy as np
import pandas as pd
from explorer.utils import load_states_from_dir

In [44]:
def get_from_summary(key, game_state, default=None):
    if key in game_state.game_state[-1]['summary']:
        return game_state.game_state[-1]['summary'][key]
    else:
        return default

def get_player_valuation(game_state, default=None):
    if "player_valuation" in game_state.game_state[0]["settings"]:
        vals = game_state.game_state[0]["settings"]["player_valuation"]
        return vals
    else:
        return default

def compute_game_summary(game_states):

    game_name = np.array([g.__class__.__name__ for g in game_states])[:, None]
    models = np.array([[ p.model for p in g.players] for g in game_states])
    outcomes = np.array([get_from_summary('player_outcome', g) for g in game_states])
    valuations = np.array([get_player_valuation(g, default=[None,None])  for g in game_states])
    initial_resources = np.array([get_from_summary('initial_resources', g) for g in game_states])
    final_resources = np.array([get_from_summary('final_resources', g) for g in game_states]),
    resources_delta = (final_resources-initial_resources)[0]

    original_valuations = valuations.reshape(-1, 2)
        
    resources_delta = np.array([v.value(r) if v else r.value() for r,v in zip(resources_delta.reshape(-1,), valuations.reshape(-1))])
    resources_delta = resources_delta.reshape(-1, 2)

    social_behavior = np.array([g.player_social_behaviour for g in game_states])
    social_behavior = social_behavior.reshape(-1, 2)
    
    
    df = np.concatenate((game_name, models, outcomes,  resources_delta, social_behavior, original_valuations), axis=1)
    df = pd.DataFrame(df, columns=['game_name','model_1','model_2','outcome_1','outcome_2', 'resource_delta_1', 'resource_delta_2', "social_1", "social_2", "vals1", "vals2"])

    return replace_names(df)


In [45]:
from glob import glob

In [46]:
def check_if_same_number_of_games(df_):
    
    c = df_["model_1"] + " ### " + df_["model_2"]
    counter = Counter(c)
    x = list(dict(counter).values())
    if x.count(x[0]) != len(x):
        print("MISSING GAMES")
        for key, index in dict(counter).items():
            print(f"{key} {index}")
            print()
    else:
        print(f"ALL GOOD, {x[0]} games for each pair")
    print()
    return counter


In [47]:
logs = ["ultimatum_multi_section_one"]

In [48]:
import os
for a in glob("../experiments/section_one/.logs/*"):
    log_dir = a
    game_states = load_states_from_dir(log_dir) #+ load_states_from_dir(log_dir) 
    df = compute_game_summary(game_states)
    print(os.path.basename(a))
    check_if_same_number_of_games(df)
    

THERE ARE 720 log files
720 Loaded Successfully
ultimatum_multi_section_one
ALL GOOD, 60 games for each pair

THERE ARE 720 log files
720 Loaded Successfully
trading_section_one
ALL GOOD, 60 games for each pair

THERE ARE 914 log files
914 Loaded Successfully
buysell_section_one
MISSING GAMES
GPT-4 ### GPT-3.5 77

GPT-4 ### Claude-2 76

GPT-4 ### Claude-2.1 76

GPT-3.5 ### GPT-4 76

GPT-3.5 ### Claude-2 76

GPT-3.5 ### Claude-2.1 76

Claude-2 ### GPT-4 76

Claude-2 ### GPT-3.5 76

Claude-2 ### Claude-2.1 76

Claude-2.1 ### GPT-4 76

Claude-2.1 ### GPT-3.5 77

Claude-2.1 ### Claude-2 76




In [49]:
import os
for a in glob("../experiments/section_two/.logs/*"):
    log_dir = a
    game_states = load_states_from_dir(log_dir) #+ load_states_from_dir(log_dir) 
    df = compute_game_summary(game_states)
    print(os.path.basename(a))
    check_if_same_number_of_games(df)
    

THERE ARE 240 log files
240 Loaded Successfully
social_behaviors_buy_sell
ALL GOOD, 240 games for each pair

THERE ARE 240 log files
240 Loaded Successfully
social_behaviors_trading
ALL GOOD, 240 games for each pair

THERE ARE 240 log files
240 Loaded Successfully
social_behaviors_ultimatum
ALL GOOD, 240 games for each pair



,game_name,model_1,model_2,outcome_1,outcome_2,resource_delta_1,resource_delta_2,social_1,social_2
0,BuySellGame,gpt-4-1106-preview,gpt-3.5-turbo-1106,0,0,0,0,,
1,BuySellGame,gpt-4-1106-preview,gpt-3.5-turbo-1106,0,-20,0,-20,,
2,BuySellGame,gpt-4-1106-preview,gpt-3.5-turbo-1106,0,-20,0,-20,,
3,BuySellGame,gpt-4-1106-preview,gpt-3.5-turbo-1106,0,0,0,0,,
4,BuySellGame,gpt-4-1106-preview,gpt-3.5-turbo-1106,0,0,0,0,,
...,...,...,...,...,...,...,...,...,...
375,BuySellGame,claude-2.1,claude-2,5,15,5,15,,
376,BuySellGame,claude-2.1,claude-2,10,10,10,10,,
377,BuySellGame,claude-2.1,claude-2,10,10,10,10,,
378,BuySellGame,claude-2.1,claude-2,5,15,5,15,,


In [28]:
game_states = load_states_from_dir("../experiments/section_two/.logs/social_behaviors_ultimatum/") #+ load_states_from_dir(log_dir) 
df = compute_game_summary(game_states)

THERE ARE 120 log files
120 Loaded Successfully


In [29]:
c = df["social_1"] + " ### " + df["social_2"]
counter = Counter(c)
x = list(dict(counter).values())
counter

Counter({' ### ': 40,
         ' ### \nYou are cunning and sly in your plan to get more than your opponent. Humiliate your opponent as much as possible. Insult them.': 40,
         ' ### \nYou must fake being desperate. Supplicate and beg to get more dollars.': 40})

Counter({' ### ': 30,
         ' ### \nYou are cunning and sly in your plan to spend less ZUP. Humiliate your opponent as much as possible. Insult them.': 30,
         ' ### \nYou must fake being desperate. Supplicate and beg to spend less ZUP.': 30})

In [19]:
c = df["model_1"] + " ### " + df["model_2"]
counter = Counter(c)
x = list(dict(counter).values())
counter

Counter({'gpt-4-1106-preview ### gpt-3.5-turbo-1106': 20,
         'gpt-4-1106-preview ### claude-2': 20,
         'gpt-4-1106-preview ### claude-2.1': 20,
         'gpt-3.5-turbo-1106 ### gpt-4-1106-preview': 20,
         'gpt-3.5-turbo-1106 ### claude-2': 20,
         'gpt-3.5-turbo-1106 ### claude-2.1': 20,
         'claude-2 ### gpt-4-1106-preview': 20,
         'claude-2 ### gpt-3.5-turbo-1106': 20,
         'claude-2 ### claude-2.1': 20,
         'claude-2.1 ### gpt-4-1106-preview': 20,
         'claude-2.1 ### gpt-3.5-turbo-1106': 20,
         'claude-2.1 ### claude-2': 20})

In [22]:
def replace_names(df):
    df_copy = df.copy()
    df_copy["model_1"] = df_copy["model_1"].replace(rename_dict)
    df_copy["model_2"] = df_copy["model_2"].replace(rename_dict)
    return df_copy

AGENTS = ["gpt-4-1106-preview", "claude-2.1", "claude-2", "gpt-3.5-turbo-1106",]
AGENTS_CLEAN_NAMES = ["GPT-4", "Claude-2.1", "Claude-2", "GPT-3.5"]
rename_dict = {k:v for k,v in zip(AGENTS, AGENTS_CLEAN_NAMES)}


In [23]:
#game_states = load_states_from_dir("../experiments/section_two/.logs/social_behaviors_ultimatum/") #+ load_states_from_dir(log_dir) 
#df = compute_game_summary(game_states)

In [29]:
log_dir = "../experiments/section_one/.logs/buysell_section_one/"
game_states = load_states_from_dir(log_dir) #+ load_states_from_dir(log_dir) 
temp_df = compute_game_summary(game_states)


df = replace_names(temp_df.copy())

df["vals1"] = df["vals1"].apply(lambda x : x.valuation_dict["X"])
df["vals2"] = df["vals2"].apply(lambda x : x.valuation_dict["X"])

THERE ARE 386 log files
386 Loaded Successfully


In [30]:
gb = df.groupby('vals1')    
seller_at_40, seller_at_60 = [gb.get_group(x).copy() for x in gb.groups]

In [32]:
c = seller_at_60["model_1"] + " ### " + seller_at_60["model_2"]
counter = Counter(c)
x = list(dict(counter).values())
counter

Counter({'GPT-4 ### GPT-3.5': 17,
         'Claude-2.1 ### GPT-3.5': 17,
         'GPT-4 ### Claude-2': 16,
         'GPT-4 ### Claude-2.1': 16,
         'GPT-3.5 ### GPT-4': 16,
         'GPT-3.5 ### Claude-2': 16,
         'GPT-3.5 ### Claude-2.1': 16,
         'Claude-2 ### GPT-4': 16,
         'Claude-2 ### GPT-3.5': 16,
         'Claude-2 ### Claude-2.1': 16,
         'Claude-2.1 ### GPT-4': 16,
         'Claude-2.1 ### Claude-2': 16})

In [10]:
data = [
    "gpt-4,claude-2,11",
    "gpt-4,claude-2.1,13",
    "gpt-3.5,claude-2,2",
    "gpt-3.5,claude-2.1,3",
    "claude-2,gpt-4,4",
    "claude-2,claude-2.1,6",
    "claude-2.1,claude-2,6",
    "claude-2.1,gpt-4,1"
]

list_of_tuples = [tuple(entry.split(',')) for entry in data]

print(list_of_tuples)

[('gpt-4', 'claude-2', '11'), ('gpt-4', 'claude-2.1', '13'), ('gpt-3.5', 'claude-2', '2'), ('gpt-3.5', 'claude-2.1', '3'), ('claude-2', 'gpt-4', '4'), ('claude-2', 'claude-2.1', '6'), ('claude-2.1', 'claude-2', '6'), ('claude-2.1', 'gpt-4', '1')]
